In [67]:
import pandas as pd
import numpy as np
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time 
from tqdm import tqdm

In [76]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

songIds = []
for i in tqdm(range(1964, 2023)):
    url = f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={i}' 
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('tbody > tr.lst50')

    for tr in trs:
        if tr.select_one('.bullet_icons.age_19'):
            continue
        songId  = tr.select_one('.btn.btn_icon_detail')['onclick'].split("'")[-2]
        songIds.append(songId)
songIds[:5],len(songIds)

100%|██████████| 59/59 [02:57<00:00,  3.01s/it]


(['4083218', '1003023', '1754987', '1002782', '1003568'], 2646)

In [77]:
df_songIds = pd.DataFrame(songIds)
df_songIds.to_csv('data/melon_year_songIds.csv', index=False)

In [ ]:
# 시대별 1964 ~ 2022 (59년)
# 2646곡에서 중복제거시 2544
# 300곡씩 파일 저장하기
# songIds가 1964~ 2022까지  set 전 2646
# set(songIds) = 2544개
  
# songIds[:300]     (247, 247)  (222, 222) 
# songIds[300:600]  (244, 242)  (237, 235) 
# songIds[600:900]  (257, 253)  (257, 253) 
# songIds[900:1200] (272, 258)  (269, 255) 
# songIds[1200:1500](273, 268)  (272, 267) 
# songIds[1500:1800](283, 281)  (283, 281) 
# songIds[1800:2100](288, 287)  (286, 285) 
# songIds[2100:2400](298, 284)  (292, 278) 
# songIds[2400:]    (245, 206)  (245, 206) 

In [79]:
def nan(data):
    if data != '-':
        return data
    else:
        return np.nan

In [90]:
infos = []
for songId in tqdm(songIds[1800:2100]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = nan(ul[0].select_one('.artist_name').get_text().strip())
        composer = nan(ul[1].select_one('.artist_name').get_text().strip())
        date = nan(info.select_one('dl > dd:nth-child(4)').get_text().replace('.',''))
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '').replace(',', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('#d_video_summary')).split('\n')[-2].replace('\t', '').replace('<br/>', '\n').replace('\n\n', '\n').strip()
        buttons = info.select('.d_djcol_list > ul > li > .thumb > a ')
        plylstSeq = nan(' '.join([button['href'].split(',\'')[-1][:-3] for button in buttons]))
        infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                    'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                    'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
    except:
        continue

100%|██████████| 300/300 [15:31<00:00,  3.11s/it]


In [91]:
df = pd.DataFrame(infos)
df.head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,1204915,Hold the line,조pd (ZoPD),랩/힙합,HOLD THE LINE,조pd (ZoPD),윤일상,20060530,https://cdnimg.melon.co.kr/cm/album/images/003...,35,17526,nu music for summer 2 double O 6 \nmost wanted...,442706386 100098263
1,1459449,까만안경 (Feat. Day Light),이루,발라드,이루 LEVEL II,윤명선,윤명선,20061009,https://cdnimg.melon.co.kr/cm/album/images/003...,106,62017,까만 안경을 써요\n아주 까만 밤인데 말이죠 \n앞이 보이지 않아도 \n괜찮아요 ...,442706386 417639180


In [92]:
df.shape[0], df.songId.nunique()

(286, 285)

In [93]:
df.to_csv('data/melon_year7.csv', index=False)
df.head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,1204915,Hold the line,조pd (ZoPD),랩/힙합,HOLD THE LINE,조pd (ZoPD),윤일상,20060530,https://cdnimg.melon.co.kr/cm/album/images/003...,35,17526,nu music for summer 2 double O 6 \nmost wanted...,442706386 100098263
1,1459449,까만안경 (Feat. Day Light),이루,발라드,이루 LEVEL II,윤명선,윤명선,20061009,https://cdnimg.melon.co.kr/cm/album/images/003...,106,62017,까만 안경을 써요\n아주 까만 밤인데 말이죠 \n앞이 보이지 않아도 \n괜찮아요 ...,442706386 417639180


In [94]:
pd.read_csv('data/melon_year7.csv', dtype=str).tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
284,3753304,벚꽃 엔딩,버스커 버스커,록/메탈,버스커 버스커 1집,장범준,장범준,20120329,https://cdnimg.melon.co.kr/cm/album/images/020...,2270,335073,그대여 그대여 그대여 \n그대여 그대여 \n오늘은 우리 같이 걸어요 이 거리를 \...,411111859 101565452
285,3837683,Loving U (러빙유),씨스타,댄스,Summer Special 'Loving U',이단옆차기,이단옆차기,20120628,https://cdnimg.melon.co.kr/cm/album/images/021...,152,79100,Hey 참 이상해 정말 strange\n이런 적 없었는데 널 볼때마다\n나 두근두근...,508457531 477814461
